In [1]:
import numpy as np
from numpy import pi
import pyregion
import graphinglib as gl
# import latexify   # only for python <= 3.11

from src.hdu.maps.map import Map

In [2]:
# @latexify.function(use_math_symbols=True)
def pdf(x, sigma, mu):
    return 1 / (np.sqrt(2*pi) * sigma) * np.exp((-(x - mu)**2) / (2*sigma**2))

# pdf

$$ \displaystyle \mathrm{pdf}(x, \sigma, \mu) = \frac{1}{\sqrt{ 2 \pi } \cdot \sigma} \cdot \exp \mathopen{}\left( \frac{-\mathopen{}\left( x - \mu \mathclose{}\right)^{2}}{2 \sigma^{2}} \mathclose{}\right) $$

# Computations of PDF histograms

In [3]:
def make_histograms(map_: Map) -> list[gl.Histogram]:
    regions = [
        None,
        pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"),
        pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"),
        pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg")
    ]

    histograms = []
    for region in regions:
        masked = map_.get_masked_region(region)
        stats = masked.data.get_statistics()
        probability_density = pdf(
            x=np.log(masked.data / stats["median"]),
            sigma=stats["stddev"],
            mu=stats["median"]
        )

        # probability_density = 1 / (np.sqrt(2*np.pi) * stats["stddev"]) \
        #                     * np.exp((-(masked.data - stats["median"])**2) / (2*stats["stddev"]**2))
        # probability_density = ((masked.data - stats["median"]) / stats["stddev"])
        bin_width = (masked.uncertainties.get_statistics()["median"] / stats["stddev"])
        histograms.append(gl.Histogram(
            data=probability_density[~np.isnan(probability_density)],
            face_color="black",
            edge_color="none",
            alpha=1,
            normalize=False,
            number_of_bins=20,
            show_params=False
            # number_of_bins=int(((np.nanmax(probability_density) + bin_width - np.nanmin(probability_density)) / bin_width)[0])
            # number_of_bins=(map_.uncertainties.get_statistics()["median"] / stats["stddev"])
        ))

    return histograms

In [13]:
names = [
    "main region",
    "diffuse region",
    "central region",
    "filament region"
]

turbulence_map = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")
density_map = Map(
    data=Map.load("summer_2023/gaussian_fitting/maps/external_maps/"
                 +"dens_it_sii_sans_fcorr_nii_plus_plusmin_pouss_seuil_errt_1000.fits").data,
    uncertainties=Map.load("summer_2023/gaussian_fitting/maps/external_maps/"
                          +"dens_it_sii_sans_fcorr_nii_plus_pluserr_pouss_seuil_errt_1000.fits").data,
    header=Map.load("summer_2023/gaussian_fitting/maps/external_maps/"
                   +"dens_it_sii_sans_fcorr_nii_plus_plusmin_pouss_seuil_errt_1000.fits").header
)
temperature_map = Map(
    data=Map.load("summer_2023/gaussian_fitting/maps/external_maps/temp_it_nii_8300.fits").data,
    uncertainties=Map.load("summer_2023/gaussian_fitting/maps/external_maps/temp_it_nii_err_8300.fits").data,
    header=Map.load("summer_2023/gaussian_fitting/maps/external_maps/temp_it_nii_8300.fits").header
)

alignment_map = (turbulence_map * 0).get_reprojection_on(density_map)

# Remove pixels that are not present in the alignment map
density_map += alignment_map
temperature_map += alignment_map

figs = []
for name, hist in zip(names, make_histograms(density_map)):
    fig = gl.Figure(
        x_label="Density [-]",
        y_label=f"PDF in {name}"
    )
    fig.add_elements(hist)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2, 2), size=(10,10))
# multifig.save("summer_2023/gaussian_fitting/figures/density/new_pdf.pdf")

figs = []
for name, hist in zip(names, make_histograms(turbulence_map)):
    fig = gl.Figure(
        x_label="Turbulence [-]",
        y_label=f"PDF in {name}"
    )
    fig.add_elements(hist)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2, 2), size=(10,10))
# multifig.save("summer_2023/gaussian_fitting/figures/turbulence/new_pdf.pdf")

figs = []
for name, hist in zip(names, make_histograms(temperature_map)):
    fig = gl.Figure(
        x_label="Temperature [-]",
        y_label=f"PDF in {name}"
    )
    fig.add_elements(hist)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2, 2), size=(10,10))
multifig.save("summer_2023/gaussian_fitting/figures/temperature/new_pdf.pdf")